# Current Model: Bayesian Model

Steps:
1. Edit MPNN model to usd Bayes_Linear and other chemprop layers

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="2"
import argparse
import time
import csv
import sys
import json
import random
import numpy as np
import pprint
import yaml
from torch import nn

import torch
import torch.multiprocessing as mp

import ray
from ray import tune

from matdeeplearn import models, process, training

config_path = 'config.yml'
#os.path.exists(config_path)
# os
os.path.abspath(os.getcwd())

assert os.path.exists(config_path), (
    "Config file not found in " + config_path
  )
with open(config_path, "r") as ymlfile:
    config = yaml.load(ymlfile, Loader=yaml.FullLoader)


In [2]:
#General imports
import csv
import os
import time
from datetime import datetime
import shutil
import copy
import numpy as np
from functools import partial
import platform
import random
import pandas as pd

##Torch imports
import torch.nn.functional as F
import torch
from torch_geometric.data import DataLoader, Dataset
from torch_geometric.nn import DataParallel
import torch_geometric.transforms as T
from torch.utils.data.distributed import DistributedSampler
from torch.nn.parallel import DistributedDataParallel
import torch.distributed as dist
import torch.multiprocessing as mp

##Matdeeplearn imports
from matdeeplearn import models
import matdeeplearn.process as process
import matdeeplearn.training as training
from matdeeplearn.models.utils import model_summary

In [3]:
config['Job'] = config['Job']['Training']
#MEGNet_demo
config["Models"] = config["Models"].get("CGCNN_demo")

In [4]:
config["Models"]

{'model': 'CGCNN',
 'dim1': 100,
 'dim2': 150,
 'pre_fc_count': 1,
 'gc_count': 4,
 'post_fc_count': 3,
 'pool': 'global_mean_pool',
 'pool_order': 'early',
 'batch_norm': 'True',
 'batch_track_stats': 'True',
 'act': 'relu',
 'dropout_rate': 0.0,
 'epochs': 250,
 'lr': 0.002,
 'batch_size': 100,
 'optimizer': 'AdamW',
 'optimizer_args': {},
 'scheduler': 'ReduceLROnPlateau',
 'scheduler_args': {'mode': 'min',
  'factor': 0.8,
  'patience': 10,
  'min_lr': 1e-05,
  'threshold': 0.0002}}

In [5]:
config["Processing"]["data_path"] = "data/pt_data/pt_data_2"
world_size = torch.cuda.device_count()
print(world_size)
rank = 'cuda'
print(world_size)
data_path = config["Processing"]["data_path"]
job_parameters= config["Job"]
training_parameters= config["Training"]
model_parameters= config["Models"]
processing_args= config['Processing']

1
1


In [6]:
##DDP
training.ddp_setup(rank, world_size)
##some issues with DDP learning rate
if rank not in ("cpu", "cuda"):
    model_parameters["lr"] = model_parameters["lr"] * world_size

##Get dataset
dataset = process.get_dataset(data_path, training_parameters["target_index"], True,  processing_args= config['Processing'])

print('Done Processing')

if rank not in ("cpu", "cuda"):
    dist.barrier()

##Set up loader
(
    train_loader,
    val_loader,
    test_loader,
    train_sampler,
    train_dataset,
    val_dataset,
    test_dataset,
) = training.loader_setup(
    training_parameters["train_ratio"],
    training_parameters["val_ratio"],
    training_parameters["test_ratio"],
    model_parameters["batch_size"],
    dataset,
    rank,
    job_parameters["seed"],
    world_size,
)







Done Processing
train length: 14850 val length: 1980 test length: 2970 unused length: 1 seed : 0


/home/varivoda/.local/lib/python3.7/site-packages/torch_geometric/deprecation.py:13: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [8]:
##Set up model
model =training.model_setup(
    rank,
    model_parameters["model"],
    model_parameters,
    dataset,
    job_parameters["load_model"],
    job_parameters["model_path"],
    model_parameters.get("print_model", True),
)

##Set-up optimizer & scheduler
optimizer = getattr(torch.optim, model_parameters["optimizer"])(
    model.parameters(),
    lr=model_parameters["lr"],
    **model_parameters["optimizer_args"]
)
scheduler = getattr(torch.optim.lr_scheduler, model_parameters["scheduler"])(
    optimizer, **model_parameters["scheduler_args"]
)

--------------------------------------------------------------------------
               Layer.Parameter    Param Tensor Shape              Param #
--------------------------------------------------------------------------
         pre_lin_list.0.weight            [100, 114]                11400
           pre_lin_list.0.bias                 [100]                  100
      conv_list.0.lin_f.weight            [100, 250]                25000
        conv_list.0.lin_f.bias                 [100]                  100
      conv_list.0.lin_s.weight            [100, 250]                25000
        conv_list.0.lin_s.bias                 [100]                  100
         conv_list.0.bn.weight                 [100]                  100
           conv_list.0.bn.bias                 [100]                  100
      conv_list.1.lin_f.weight            [100, 250]                25000
        conv_list.1.lin_f.bias                 [100]                  100
      conv_list.1.lin_s.weight      

In [7]:
"""loss_func = neg_log_like 
metric_func = get_metric_func(metric=args.metric)
num_workers = 0"""

'loss_func = neg_log_like \nmetric_func = get_metric_func(metric=args.metric)\nnum_workers = 0'

In [29]:
model

CGCNN(
  (pre_lin_list): ModuleList(
    (0): Linear(in_features=114, out_features=100, bias=True)
  )
  (conv_list): ModuleList(
    (0): CGConv(100, dim=50)
    (1): CGConv(100, dim=50)
    (2): CGConv(100, dim=50)
    (3): CGConv(100, dim=50)
  )
  (bn_list): ModuleList(
    (0): DiffGroupNorm(100, groups=10)
    (1): DiffGroupNorm(100, groups=10)
    (2): DiffGroupNorm(100, groups=10)
    (3): DiffGroupNorm(100, groups=10)
  )
  (post_lin_list): ModuleList(
    (0): Linear(in_features=100, out_features=150, bias=True)
    (1): Linear(in_features=150, out_features=150, bias=True)
    (2): Linear(in_features=150, out_features=150, bias=True)
  )
  (lin_out): Linear(in_features=150, out_features=1, bias=True)
)

In [9]:
rho_min_bbp = -5.5
rho_max_bbp = -5
for module in model.children():
    if isinstance(module, torch.nn.modules.container.ModuleList):
        for layer in module:
            if isinstance(layer,models.mpnn_bayes.BayesLinear):
                layer.init_rho(rho_min_bbp, rho_max_bbp)
                print("layer")
    if isinstance(module,models.mpnn_bayes.BayesLinear):
        module.init_rho(rho_min_bbp, rho_max_bbp)
        print("module")

layer
layer
layer
layer
module


In [9]:
for data in train_loader:
    mydat = data.to(rank)
    break

In [10]:
model = model.to(rank)

In [12]:
dir(mydat)

['__call__',
 '__cat_dim__',
 '__class__',
 '__contains__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__inc__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setitem__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_store',
 'apply',
 'apply_',
 'batch',
 'clone',
 'coalesce',
 'contains_isolated_nodes',
 'contains_self_loops',
 'contiguous',
 'cpu',
 'cuda',
 'debug',
 'detach',
 'detach_',
 'edge_attr',
 'edge_index',
 'edge_stores',
 'from_data_list',
 'from_dict',
 'get',
 'has_isolated_nodes',
 'has_self_loops',
 'index_select',
 'is_coalesced',
 'is_directed',
 'is_undirected',
 'keys',
 'node_stores',
 'num_edge_featu

In [13]:
preds, kl = model(mydat, sample = True)

In [14]:
dataset[0

Data(edge_index=[2, 100], edge_weight=[100], y=-41.396141052246094, z=[10], u=[1, 3], structure_id=[1], x=[10, 114], edge_attr=[100, 50])

In [30]:
#loss_method = training_parameters['loss']
loss_method = 'nll_loss'

In [31]:
train_size = len(train_dataset.indices)

In [35]:
preds.dim()

1

In [32]:
optimizer.zero_grad()
data_loss = getattr(F, loss_method)(preds, mydat.y) 

ValueError: Expected 2 or more dimensions (got 1)

In [18]:
total_loss = data_loss + kl

In [19]:
total_loss.retain_grad()
total_loss.backward()

In [20]:
total_loss.grad

tensor(1., device='cuda:0')

In [22]:
total_loss

tensor(119810.6250, device='cuda:0', grad_fn=<AddBackward0>)

In [ ]:
def train_bbp(model, optimizer, loader, loss_method, rank, my_coeff = 1, switch = None, train_size):
    model.train()
    loss_all = 0
    data_loss_sum = 0
    kl_loss_sum = 0
    kl_loss_depth_sum = 0
    count = 0
    for data in loader:
        data = data.to(rank)
        optimizer.zero_grad()
        output, kl_loss = model(data, sample=True)
        kl_loss /= train_size
        data_loss = getattr(F, loss_method)(output, data.y, torch.exp(model.log_noise)) 
        loss.backward()
        loss_all += loss.detach() * output.size(0)
        optimizer.step()
        count = count + output.size(0)
            

        # clip = 10
        # torch.nn.utils.clip_grad_norm_(model.parameters(), 10)

        

    loss_all = loss_all / count
    return loss_all

"""
        data_loss_cum = 0
        kl_loss_cum = 0

        for i in range(args.samples_bbp):
            preds, kl_loss_i = model(mol_batch, features_batch, sample=True)
            data_loss_i = loss_func(preds, targets, torch.exp(model.log_noise))                    
            kl_loss_i /= args.train_data_size                    

            data_loss_cum += data_loss_i
            kl_loss_cum += kl_loss_i

        data_loss = data_loss_cum / args.samples_bbp
        kl_loss = kl_loss_cum / args.samples_bbp
        
"""

In [ ]:
### bbp non sample option

#our switch should be 2 and 5 samples
if bbp_switch == 1:    
    preds, kl_loss = model(mol_batch, features_batch, sample = False)
    data_loss = loss_func(preds, targets, torch.exp(model.log_noise))
    kl_loss /= args.train_data_size
    loss = data_loss + kl_loss  

### bbp sample option
if bbp_switch == 2:

    if args.samples_bbp == 1:
        preds, kl_loss = model(mol_batch, features_batch, sample=True)
        data_loss = loss_func(preds, targets, torch.exp(model.log_noise))
        kl_loss /= args.train_data_size

    elif args.samples_bbp > 1:
        data_loss_cum = 0
        kl_loss_cum = 0

        for i in range(args.samples_bbp):
            preds, kl_loss_i = model(mol_batch, features_batch, sample=True)
            data_loss_i = loss_func(preds, targets, torch.exp(model.log_noise))                    
            kl_loss_i /= args.train_data_size                    

            data_loss_cum += data_loss_i
            kl_loss_cum += kl_loss_i

        data_loss = data_loss_cum / args.samples_bbp
        kl_loss = kl_loss_cum / args.samples_bbp

    loss = data_loss + kl_loss

In [36]:
import matplotlib.pyplot as plt
import numpy as np

from BNNLayer import BNNLayer
from BNN import BNN

import torch
from torch.autograd import Variable

In [ ]:
def train(model, optimizer, loader, loss_method, rank, my_coeff = 1):
    model.train()
    loss_all = 0
    count = 0
    for data in loader:
        data = data.to(rank)
        optimizer.zero_grad()
        output = model(data)
        # print(data.y.shape, output.shape)
        if loss_method == "evidential":
            loss =  evidential_regresssion_loss(data.y, output, my_coeff)
            loss.backward()
            loss_all += loss.detach() * torch.flatten(output[0]).size(0)
            optimizer.step()
            count = count + torch.flatten(output[0]).size(0)
        else:
            loss = getattr(F, loss_method)(output, data.y)
            loss.backward()
            loss_all += loss.detach() * output.size(0)
            optimizer.step()
            count = count + output.size(0)

        # clip = 10
        # torch.nn.utils.clip_grad_norm_(model.parameters(), 10)

        

    loss_all = loss_all / count
    return loss_all


In [105]:
for data in test_loader:
        data = data.to(rank)
        optimizer.zero_grad()
        output = model(data)
        print(data.x.shape,data.y.shape, output.shape)

torch.Size([1000, 114]) torch.Size([100]) torch.Size([100])
torch.Size([500, 114]) torch.Size([50]) torch.Size([50])


In [7]:
for data in train_loader:
    data = data.to(rank)
    my_dat = data
    break

In [12]:
output = model(my_dat)

In [14]:
output.shape

torch.Size([1000, 114])

In [10]:
my_dat.x.shape

torch.Size([1152, 114])

In [52]:
##### Initialize network
bnn = BNN(BNNLayer(114, 100, activation='relu', prior_mean=0, prior_rho=0),
          BNNLayer(100, 1, activation='none', prior_mean=0, prior_rho=0))

optim = torch.optim.Adam(bnn.parameters(), lr=1e-1)

# Main training loop
for epoch in range(50):
    for data in train_loader:
        kl, lg_lklh = bnn.Forward(data.x, data.y, 5, 'Gaussian')
        loss = BNN.loss_fn(kl, lg_lklh, 1)
        optim.zero_grad()
        loss.backward()
        optim.step()



In [68]:
my_dat.x

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [67]:
for data in test_loader:
    mydat_2 = data
    break
mydat_2.x

tensor([-40.6112, -39.0617, -41.3328, -39.4992, -41.4644, -40.3276, -40.2760,
        -41.8916, -39.3135, -40.9401, -41.3470, -40.5656, -38.2606, -42.3238,
        -41.7147, -39.5980, -42.6903, -39.4735, -36.6092, -42.2160, -39.0374,
        -40.2563, -39.9356, -38.0120, -38.9859, -38.8221, -39.0965, -39.5741,
        -40.8805, -40.4843, -38.6084, -41.0291, -41.0326, -42.5290, -40.9047,
        -40.2822, -41.1662, -40.1886, -41.4339, -39.9165, -41.5909, -40.1056,
        -42.6482, -41.4383, -40.8408, -41.6736, -42.4117, -41.1575, -39.8198,
        -40.6107, -39.8405, -40.1046, -40.7631, -38.9896, -40.2281, -42.5694,
        -42.3701, -40.2304, -39.3464, -40.4437, -40.9188, -41.4603, -42.4589,
        -39.7174, -39.6986, -41.3970, -41.3076, -42.7034, -38.1881, -39.5739,
        -42.7183, -41.3451, -41.8559, -40.6139, -40.4188, -41.3961, -41.4384,
        -39.9930, -41.2173, -38.0770, -41.0816, -38.9201, -41.4408, -39.3002,
        -39.0565, -41.5707, -39.9852, -41.2182, -39.2941, -39.09

In [74]:
pred_lst = [bnn.forward(mydat_2.x, mode='MC').data.numpy().squeeze(1) for _ in range(100)]
pred = np.array(pred_lst).T
pred_mean = pred.mean(axis=1)
pred_std = pred.std(axis=1)

In [76]:
pred_mean

array([-40.76368 , -40.69465 , -40.65186 , -40.68389 , -40.50751 ,
       -40.649426, -40.512314, -40.567337, -40.50829 , -40.854534,
       -40.478046, -40.615677, -40.5848  , -40.542377, -40.6007  ,
       -40.582447, -40.624035, -40.514698, -40.416447, -40.4691  ,
       -40.814247, -40.480526, -40.61916 , -40.576405, -40.765404,
       -40.578148, -40.754173, -40.54605 , -40.59684 , -40.960327,
       -40.35377 , -40.466003, -40.67522 , -40.68209 , -40.56576 ,
       -40.46866 , -40.60142 , -40.38169 , -40.3964  , -40.481373,
       -40.521656, -40.51847 , -40.721798, -40.51322 , -40.547752,
       -40.479427, -40.741985, -40.649952, -40.6453  , -40.736725,
       -40.40797 , -40.815025, -40.425964, -40.48258 , -40.656494,
       -40.520332, -40.62672 , -40.741653, -40.436344, -40.78968 ,
       -40.605133, -40.745686, -40.486153, -40.436836, -40.658764,
       -40.5975  , -40.606346, -40.52455 , -40.55612 , -40.66819 ,
       -40.81333 , -40.6959  , -40.478367, -40.492344, -40.474

In [77]:
pred_std

array([1.0547813 , 1.2954718 , 1.1567339 , 1.1680073 , 1.1681857 ,
       1.0926514 , 1.2417028 , 1.2096938 , 1.0787663 , 1.208754  ,
       1.0554477 , 1.2546116 , 1.0570738 , 1.0812312 , 1.3572689 ,
       1.1870748 , 1.09098   , 1.3453679 , 0.96389604, 1.3075686 ,
       1.1034024 , 1.1449726 , 1.0919079 , 1.0964007 , 1.2822737 ,
       1.1204354 , 1.3033876 , 1.178195  , 1.0792165 , 1.1797916 ,
       1.2000948 , 1.2065288 , 1.1125236 , 1.1267766 , 1.2139312 ,
       1.1807195 , 1.2163414 , 1.14374   , 1.0159215 , 1.123853  ,
       1.0892687 , 1.1751349 , 1.2055798 , 1.2320247 , 1.0799826 ,
       1.1605153 , 1.2267975 , 1.1553559 , 1.2382569 , 1.2924211 ,
       1.1440053 , 1.1851903 , 1.2156494 , 1.0019516 , 1.166209  ,
       1.2151327 , 1.0975863 , 1.1164252 , 1.0995458 , 1.1612872 ,
       1.1375529 , 1.1420051 , 1.3190334 , 1.3501195 , 1.2169507 ,
       1.0831354 , 1.2276876 , 1.2129567 , 1.01441   , 1.1461436 ,
       1.0903529 , 1.223599  , 1.1849741 , 1.3343154 , 1.07363

In [ ]:
test_X = Variable(test_set.test_data.view(test_size, -1).type(torch.FloatTensor))
test_Y = Variable(test_set.test_labels.view(test_size, -1))

pred_class = bnn.forward(test_X, mode='MAP').data.numpy().argmax(axis=1)
true_class = test_Y.data.numpy().ravel()

In [89]:
# Plotting
plt.scatter(x, y, c='navy', label='target')

x_ = np.linspace(-5, 5)
y_ = x_ ** 3
X_ = Var(x_).unsqueeze(1)

pred_lst = [bnn.forward(X_, mode='MC').data.numpy().squeeze(1) for _ in range(100)]




NameError: name 'x' is not defined

In [97]:
my_dat.x.shape

torch.Size([1000, 114])

In [98]:
pred_mean.x.shape

(1000,)

In [85]:
mydat_2.y.shape

torch.Size([100])

In [ ]:
pred = np.array(pred_lst).T
pred_mean = pred.mean(axis=1)
pred_std = pred.std(axis=1)


plt.plot(mydat_2.x, pred_mean, c='royalblue', label='mean pred')
plt.fill_between(mydat_2.x, pred_mean - 3 * pred_std, pred_mean + 3 * pred_std,
                 color='cornflowerblue', alpha=.5, label='+/- 3 std')

plt.plot(mydat_2.x, mydat_2.y, c='grey', label='truth')

plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
model.eval()
loss_all = 0
count = 0
for data in test_loader:
    data = data.to(rank)
    with torch.no_grad():
        pred_lst = [bnn.forward(data.x, mode='MC').data.numpy().squeeze(1) for _ in range(100)]
        pred = np.array(pred_lst).T
        pred_mean = pred.mean(axis=1)
        pred_std = pred.std(axis=1)
        
        
        if out == True:
            if count == 0:
                ids = [item for sublist in data.structure_id for item in sublist]
                ids = [item for sublist in ids for item in sublist]
                predict = output.data.cpu().numpy()
                target = data.y.cpu().numpy()
            else:
                ids_temp = [
                    item for sublist in data.structure_id for item in sublist
                ]
                ids_temp = [item for sublist in ids_temp for item in sublist]
                ids = ids + ids_temp
                predict = np.concatenate(
                    (predict, output.data.cpu().numpy()), axis=0
                )
                target = np.concatenate((target, data.y.cpu().numpy()), axis=0)

        count = count + output.size(0)

loss_all = loss_all / count

if out == True:
    test_out = np.column_stack((ids, target, predict))
    return loss_all, test_out

# Current Model: Deep Evidential Regression

Steps:
1. Train regular model on data set 
2. Take the last layer from the model as the output as it outputs a 1-d vector
3. Feed the vector into a gaussian regression model
4. Output a mean and variance to create confidence intervals

In [ ]:
import os
import argparse
import time
import csv
import sys
import json
import random
import numpy as np
import pprint
import yaml

import torch
import torch.multiprocessing as mp

import ray
from ray import tune

from matdeeplearn import models, process, training

In [ ]:
config_path = 'config.yml'
#os.path.exists(config_path)
# os
os.path.abspath(os.getcwd())

In [ ]:
assert os.path.exists(config_path), (
    "Config file not found in " + config_path
  )
with open(config_path, "r") as ymlfile:
    config = yaml.load(ymlfile, Loader=yaml.FullLoader)
      

In [ ]:
config["Job"] = config["Job"]['Inductive_Conformal']
config["Job"]





In [ ]:
config["Processing"]["data_path"]





In [ ]:
config["Models"] = config["Models"].get("MEGNet_demo")





In [ ]:
##Update config values from command line
run_mode = config["Job"].get("run_mode")
#config["Job"] = config["Job"].get('Training')
print(config["Job"])

In [ ]:
world_size = torch.cuda.device_count()
print(world_size)
config["Processing"]["data_path"] = "data/pt_data/pt_data_2"

In [ ]:
#os.environ["CUDA_VISIBLE_DEVICES"] = '1,2'

#torch.cuda.device_count()

In [ ]:
world_size = torch.cuda.device_count()
print(world_size)
config["Processing"]["data_path"] = "data/test_data/test_data_2"
training_parameters = config["Training"]
data_path = config["Processing"]["data_path"]
dataset = process.get_dataset(data_path, training_parameters["target_index"], False, processing_args= config['Processing'])
model_parameters = config["Models"]
job_parameters = config["Job"]
rank = "cuda"


(
    train_loader,
    val_loader,
    test_loader,
    train_sampler,
    train_dataset,
    val_dataset,
    test_dataset,
) = training.loader_setup(
    training_parameters["train_ratio"],
    training_parameters["val_ratio"],
    training_parameters["test_ratio"],
    model_parameters["batch_size"],
    dataset,
    rank,
    job_parameters["seed"],
    world_size,
)


In [ ]:
model = training.model_setup(
        rank,
        model_parameters["model"],
        model_parameters,
        dataset,
        job_parameters["load_model"],
        job_parameters["model_path"],
        model_parameters.get("print_model", True),
    ) 

  ##Set-up optimizer & scheduler
optimizer = getattr(torch.optim, model_parameters["optimizer"])(
    model.parameters(),
    lr=model_parameters["lr"],
    **model_parameters["optimizer_args"]
)
scheduler = getattr(torch.optim.lr_scheduler, model_parameters["scheduler"])(
    optimizer, **model_parameters["scheduler_args"]
)

##Start training
model = training.trainer(
    rank,
    world_size,
    model,
    optimizer,
    scheduler,
    training_parameters["loss"],
    train_loader,
    val_loader,
    train_sampler,
    model_parameters["epochs"],
    training_parameters["verbosity"],
    "my_model_temp.pth",
)

In [ ]:
train_error, train_out = training.evaluate(
    train_loader, model, training_parameters["loss"], rank, out=True
)

In [ ]:
train_out

# Getting a Frequency Distribution of Training Indices

In [ ]:
job_parameters = config['Job']

In [ ]:
my_split = job_parameters["model_n"].find('.')

In [ ]:
import pandas as pd

In [ ]:
my_ids = []

for i in range(0,50):
    df = pd.read_csv(os.path.join(job_parameters["model_path"],
                         "Bootstrap_Pred_" + 
                         str(i) + '_train_outputs.csv'))
    temp_ids = df['ids'].tolist()
    my_ids = my_ids + temp_ids


In [ ]:
my_ids = []

for i in range(0,50):
    df = pd.read_csv(os.path.join(job_parameters["model_path"],
                         "Bootstrap_Pred_" + 
                         str(i) + '_train_outputs.csv'))
    temp_ids = df['ids'].tolist()
    my_ids = my_ids + temp_ids

frame_dict = {'ids': np.unique(my_ids).tolist(), 'count': np.repeat(0,len(np.unique(my_ids).tolist()))}
freq = pd.DataFrame(frame_dict)

for item in my_ids: 
    if (item in freq['ids'].tolist()): 
        freq['count'][freq['ids'] == item] += 1
    else: 
        print('oopsie')

In [ ]:
freq.iloc[1,]

In [ ]:
import csv
with open("bootstrap_freq_training_ids_check.csv", "w") as f:

    csvwriter = csv.writer(f)
    for i in range(0,  len(my_ids)+1):
        if i == 0:
            csvwriter.writerow(
                [
                    "ids",
                    'count'
                ]
            )
        elif i > 0:
            csvwriter.writerow(freq.iloc[i - 1, :])

In [ ]:
for item in my_ids: 
    if (item in freq['ids'].tolist()): 
        freq['count'][freq['ids'] == item] += 1
    else: 
        print('oopsie')
        
        

In [ ]:
config["Job"]["parallel"] = "True"

In [ ]:
config["Processing"]["data_path"] = 'data/pt_data/pt_data_2'

In [ ]:
print("Starting Inductive Conformal Error training")
print(
    "running for "
    + str(config["Models"]["epochs"])
    + " epochs"
    + " on "
    + str(config["Job"]["model"])
    + " model" 
    )
world_size = torch.cuda.device_count()
if world_size == 0:
    print("Running on CPU - this will be slow")

    training.train_ic_errors(
        "cpu",
        world_size,
        config["Processing"]["data_path"],
        config["Job"],
        config["Training"],
        config["Models"],
        config['Processing']
    )


elif world_size > 0:
    if config["Job"]["parallel"] == "True":
        print("Running on", world_size, "GPUs")
        mp.spawn(
            training.train_ic_errors,
            args=(
                world_size,
                config["Processing"]["data_path"],
                config["Job"],
                config["Training"],
                config["Models"],
                config['Processing'],
            ),
            nprocs=world_size,
            join=True,
        )
    if config["Job"]["parallel"] == "False":
        print("Running on one GPU")
        training.train_ic_errors(
            "cuda",
            world_size,
            config["Processing"]["data_path"],
            config["Job"],
            config["Training"],
            config["Models"],
            config['Processing']
        )   

# Inductive Conformal (Hopefully this works)

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1,2"
import argparse
import time
import csv
import sys
import json
import random
import numpy as np
import pprint
import yaml

import torch
import torch.multiprocessing as mp

import ray
from ray import tune

from matdeeplearn import models, process, training

config_path = 'config.yml'
#os.path.exists(config_path)
# os
os.path.abspath(os.getcwd())

assert os.path.exists(config_path), (
    "Config file not found in " + config_path
  )
with open(config_path, "r") as ymlfile:
    config = yaml.load(ymlfile, Loader=yaml.FullLoader)
config["Job"] = config["Job"]['Inductive_Conformal']
config["Models"] = config["Models"].get("MEGNet_demo")
world_size = torch.cuda.device_count()
print(world_size)
config["Processing"]["data_path"] = "data/pt_data/pt_data_2"

In [ ]:
rank = 'cuda'
print(world_size)
data_path = config["Processing"]["data_path"]
job_parameters= config["Job"]
training_parameters= config["Training"]
model_parameters= config["Models"]
processing_args= config['Processing']

In [ ]:
##General imports
import csv
import os
import time
from datetime import datetime
import shutil
import copy
import numpy as np
from functools import partial
import platform
import random
import pandas as pd

##Torch imports
import torch.nn.functional as F
import torch
from torch_geometric.data import DataLoader, Dataset
from torch_geometric.nn import DataParallel
import torch_geometric.transforms as T
from torch.utils.data.distributed import DistributedSampler
from torch.nn.parallel import DistributedDataParallel
import torch.distributed as dist
import torch.multiprocessing as mp

##Matdeeplearn imports
from matdeeplearn import models
import matdeeplearn.process as process
import matdeeplearn.training as training
from matdeeplearn.models.utils import model_summary

In [ ]:
##DDP
training.ddp_setup(rank, world_size)
##some issues with DDP learning rate
if rank not in ("cpu", "cuda"):
    model_parameters["lr"] = model_parameters["lr"] * world_size

##Get dataset
dataset = process.get_dataset(data_path, training_parameters["target_index"], False)

print('Done Processing')

if rank not in ("cpu", "cuda"):
    dist.barrier()

##Set up loader
(
    train_loader,
    val_loader,
    test_loader,
    train_sampler,
    train_dataset,
    val_dataset,
    test_dataset,
) = training.loader_setup(
    training_parameters["train_ratio"],
    training_parameters["val_ratio"],
    training_parameters["test_ratio"],
    model_parameters["batch_size"],
    dataset,
    rank,
    job_parameters["seed"],
    world_size,
)

##Set up model
model =training.model_setup(
    rank,
    model_parameters["model"],
    model_parameters,
    dataset,
    job_parameters["load_model"],
    job_parameters["model_path"],
    model_parameters.get("print_model", True),
)

##Set-up optimizer & scheduler
optimizer = getattr(torch.optim, model_parameters["optimizer"])(
    model.parameters(),
    lr=model_parameters["lr"],
    **model_parameters["optimizer_args"]
)
scheduler = getattr(torch.optim.lr_scheduler, model_parameters["scheduler"])(
    optimizer, **model_parameters["scheduler_args"]
)

##Start training
model = training.trainer(
    rank,
    world_size,
    model,
    optimizer,
    scheduler,
    training_parameters["loss"],
    train_loader,
    val_loader,
    train_sampler,
    model_parameters["epochs"],
    training_parameters["verbosity"],
    "my_model_temp.pth",
)

if rank in (0, "cpu", "cuda"):

    train_error = val_error = test_error = float("NaN")

    ##workaround to get training output in DDP mode
    ##outputs are slightly different, could be due to dropout or batchnorm?
    train_loader = DataLoader(
        train_dataset,
        batch_size=model_parameters["batch_size"],
        shuffle=False,
        num_workers=0,
        pin_memory=True,
    )

    test_loader = training.DataLoader(
                    test_dataset,
                    batch_size=model_parameters["batch_size"],
                    shuffle=False,
                    num_workers=0,
                    pin_memory=True,
                )

    val_loader = training.DataLoader(
                    val_dataset,
                    batch_size=model_parameters["batch_size"],
                    shuffle=False,
                    num_workers=0,
                    pin_memory=True,
                )

    ##Get train error in eval mode
    train_error, train_out = training.evaluate(
        train_loader, model, training_parameters["loss"], rank, out=True
    )
    print("Train Error: {:.5f}".format(train_error))

    ##Get val error
    if val_loader != None:
        val_error, val_out = training.evaluate(
            val_loader, model, training_parameters["loss"], rank, out=True
        )
        print("Val Error: {:.5f}".format(val_error))

    ##Get test error
    if test_loader != None:
        test_error, test_out = training.evaluate(
            test_loader, model, training_parameters["loss"], rank, out=True
        )
        print("Test Error: {:.5f}".format(test_error))
        
        
        
        
        
        
        
        
        
        
        
        

In [ ]:
if job_parameters["save_model"] == "True":

    if rank not in ("cpu", "cuda"):
        torch.save(
            {
                "model_state_dict": model.state_dict(),
                "optimizer_state_dict": optimizer.state_dict(),
                "scheduler_state_dict": scheduler.state_dict(),
                "full_model": model,
            },
            job_parameters["model_path"],
        )
    else:
        torch.save(
            {
                "model_state_dict": model.state_dict(),
                "optimizer_state_dict": optimizer.state_dict(),
                "scheduler_state_dict": scheduler.state_dict(),
                "full_model": model,
            },
            job_parameters["model_path"],
        )
target_train = pd.DataFrame(train_out, columns=['index', 'target', 'predicted'])
target_val = pd.DataFrame(val_out, columns=['index', 'target', 'predicted']) 
target_test = pd.DataFrame(test_out, columns=['index', 'target', 'predicted']) 
target_errors = pd.concat([target_train,target_val,target_test], axis = 0)
target_errors = target_errors.sort_values(list(target_errors), ascending=True)
target_errors['error'] = np.absolute(target_errors['target'].apply(float) - target_errors['predicted'].apply(float))
target_errors[['index','error']].to_csv(os.path.join(os.getcwd(),data_path,'error_targets.csv'), index = False, header=False)



In [ ]:
names = ['train']*len(train_dataset.indices) + ['val']*len(val_dataset.indices) + ['test']*len(test_dataset.indices)
my_indices = pd.DataFrame({'ind_type': names, 'inidex':train_dataset.indices+val_dataset.indices+test_dataset.indices})
my_indices.to_csv(os.path.join(os.getcwd(),'data_indices_ic.csv'), index = False, header=False)

In [ ]:
my_indices

In [ ]:
new_data = process.get_dataset_error(data_path ,training_parameters["target_index"], False, processing_args)
error_train_subset =  torch.utils.data.Subset(new_data, train_dataset.indices)
error_val_subset = torch.utils.data.Subset(new_data, val_dataset.indices)
error_test_subset = torch.utils.data.Subset(new_data, test_dataset.indices)

train_loader_e = training.DataLoader(
    error_train_subset,
    batch_size=model_parameters["batch_size"],
    shuffle=False,
    num_workers=0,
    pin_memory=True,
)

val_loader_e = training.DataLoader(
                error_val_subset,
                batch_size=model_parameters["batch_size"],
                shuffle=False,
                num_workers=0,
                pin_memory=True,
            )

test_loader_e = training.DataLoader(
                error_test_subset,
                batch_size=model_parameters["batch_size"],
                shuffle=False,
                num_workers=0,
                pin_memory=True,
            )

model_errors = training.model_setup(
        rank,
        model_parameters["model"],
        model_parameters,
        new_data,
        job_parameters["load_model"],
        job_parameters["model_path"],
        model_parameters.get("print_model", True),
    ) 

In [ ]:
optimizer = getattr(torch.optim, model_parameters["optimizer"])(
    model_errors.parameters(),
    lr=model_parameters["lr"],
    **model_parameters["optimizer_args"]
)
scheduler = getattr(torch.optim.lr_scheduler, model_parameters["scheduler"])(
    optimizer, **model_parameters["scheduler_args"]
)

##Start training
model_errors = training.trainer(
    rank,
    world_size,
    model_errors,
    optimizer,
    scheduler,
    training_parameters["loss"],
    train_loader_e,
    val_loader_e,
    train_sampler,
    model_parameters["epochs"],
    training_parameters["verbosity"],
    "my_model_error_temp.pth",
)



In [ ]:
train_error_e, train_out_e = training.evaluate(
    train_loader_e, model_errors, training_parameters["loss"], rank, out=True)

val_error_e, val_out_e = training.evaluate(
    val_loader_e, model_errors, training_parameters["loss"], rank, out=True)

test_error_e, test_out_e = training.evaluate(
    test_loader_e, model_errors, training_parameters["loss"], rank, out=True)


target_train_e = pd.DataFrame(train_out_e, columns=['index', 'target_error', 'predicted_error'])
target_val_e = pd.DataFrame(val_out_e, columns=['index', 'target_error', 'predicted_error']) 
target_test_e = pd.DataFrame(test_out_e, columns=['index', 'target_error', 'predicted_error']) 
target_errors_e = pd.concat([target_train_e,target_val_e,target_test_e], axis = 0)
target_errors_e = target_errors_e.sort_values(list(target_errors_e), ascending=True)
target_errors_e['error_2'] = np.absolute(target_errors_e['target_error'].apply(float) - target_errors_e['predicted_error'].apply(float))

target_val_e_2 = copy.copy(target_val_e)
target_val_e_2['target_error'] = target_val_e_2['target_error'].apply(float)
target_val_e_2['predicted_error'] = target_val_e_2['predicted_error'].apply(float)
target_val_e_2['alpha'] = target_val_e_2['target_error']/target_val_e_2['predicted_error']

target_val_e_2 = target_val_e_2.sort_values(['alpha'], axis=0, ascending=True)
alpha = np.percentile(target_val_e_2['alpha'], 95)

target_train_e['predicted_error'] = target_train_e['predicted_error'].apply(float)
target_train_e['lower_error_confidence_level'] = target_train_e['predicted_error'] - alpha
target_train_e['upper_error_confidence_level'] = target_train_e['predicted_error'] + alpha

target_val_e['predicted_error'] = target_val_e['predicted_error'].apply(float)
target_val_e['lower_error_confidence_level'] = target_val_e['predicted_error'] - alpha
target_val_e['upper_error_confidence_level'] = target_val_e['predicted_error'] + alpha

target_test_e['predicted_error'] = target_test_e['predicted_error'].apply(float)
target_test_e['lower_error_confidence_level'] = target_test_e['predicted_error'] - alpha
target_test_e['upper_error_confidence_level'] = target_test_e['predicted_error'] + alpha


target_train_e.to_csv(os.path.join(os.getcwd(),'error_prediction_conf_train.csv'), index = False, header=False)
target_val_e.to_csv(os.path.join(os.getcwd(),'error_prediction_conf_val.csv'), index = False, header=False)
target_test_e.to_csv(os.path.join(os.getcwd(),'error_prediction_conf_test.csv'), index = False, header=False)

        ##Write outputs
if job_parameters["write_output"] == "True":

    training.write_results(
        train_out, str(job_parameters["job_name"]) + "_train_outputs.csv"
    )
    if val_loader != None:
        training.write_results(
            val_out, str(job_parameters["job_name"]) + "_val_outputs.csv"
        )
    if test_loader != None:
        training.write_results(
            test_out, str(job_parameters["job_name"]) + "_test_outputs.csv"
        )

if rank not in ("cpu", "cuda"):
    dist.destroy_process_group()

##Write out model performance to file
error_values = np.array((train_error.cpu(), val_error.cpu(), test_error.cpu()))
if job_parameters.get("write_error") == "True":
    np.savetxt(
        job_parameters["job_name"] + "_errorvalues.csv",
        error_values[np.newaxis, ...],
        delimiter=",",
    )